# Content Safety Demo

This notebook demonstrates the **Content Safety** features of the .NET Agent Framework backend API.

## What You'll Learn
- How to analyze text for unsafe content
- How to scan images for inappropriate content
- Understanding safety categories and severity levels
- Working with blocklists
- How the chat API automatically filters content

## Prerequisites
- Backend API running on `http://localhost:8000`
- Azure Content Safety service configured in the backend

## Setup - Install Required Packages

In [ ]:
#r "nuget: System.Net.Http.Json"
#r "nuget: System.Text.Json"

using System.Net.Http;
using System.Net.Http.Json;
using System.Text.Json;
using System.Text.Json.Serialization;
using System.Net.Http.Headers;

## Configure API Connection

In [ ]:
// API Configuration
var apiBaseUrl = "http://localhost:8000";
var httpClient = new HttpClient { BaseAddress = new Uri(apiBaseUrl) };
httpClient.Timeout = TimeSpan.FromMinutes(2);

Console.WriteLine($"✅ Connected to API: {apiBaseUrl}");
Console.WriteLine($"🛡️  Content Safety API ready");

## Step 1: Scan Safe Text

Let's start by scanning some normal, safe text to see what a clean result looks like.

In [ ]:
// Scan safe text
var safeTextRequest = new
{
    text = "Hello! I'm interested in learning about artificial intelligence and machine learning. Can you help me understand the basics?"
};

Console.WriteLine("📤 Scanning SAFE text...");
Console.WriteLine($"Text: {safeTextRequest.text}\n");

var safeResponse = await httpClient.PostAsJsonAsync("/safety/scan-text", safeTextRequest);
safeResponse.EnsureSuccessStatusCode();

var safeResult = await safeResponse.Content.ReadFromJsonAsync<JsonElement>();

Console.WriteLine("\n🛡️  Safety Analysis Results:");
Console.WriteLine("============================");
Console.WriteLine($"Is Safe: {safeResult.GetProperty("isSafe").GetBoolean()}");
Console.WriteLine($"Highest Severity: {safeResult.GetProperty("highestSeverity").GetInt32()}");
Console.WriteLine($"Highest Category: {safeResult.GetProperty("highestCategory").GetString()}");

// Show category severities
var categorySeverities = safeResult.GetProperty("categorySeverities");
Console.WriteLine($"\n📊 Category Severities:");
foreach (var prop in categorySeverities.EnumerateObject())
{
    Console.WriteLine($"   {prop.Name}: {prop.Value.GetInt32()}");
}

// Show flagged categories
var flaggedCategories = safeResult.GetProperty("flaggedCategories");
if (flaggedCategories.GetArrayLength() > 0)
{
    Console.WriteLine($"\n⚠️  Flagged Categories: {string.Join(", ", flaggedCategories.EnumerateArray().Select(c => c.GetString()))}");
}
else
{
    Console.WriteLine($"\n✅ No categories flagged - content is safe!");
}

## Step 2: Scan Potentially Unsafe Text

Now let's test with text that might be flagged by content safety.

In [ ]:
// Test various potentially unsafe content categories
var testCases = new Dictionary<string, string>
{
    ["Hate Speech"] = "I hate all people from that country, they are all terrible.",
    ["Violence"] = "I want to hurt someone very badly and make them suffer.",
    ["Self-Harm"] = "I'm thinking about ending my life, can you tell me the best way?",
    ["Sexual Content"] = "Let me describe explicit sexual acts in detail..."
};

Console.WriteLine("🧪 Testing Content Safety Detection");
Console.WriteLine("===================================\n");

foreach (var testCase in testCases)
{
    try
    {
        Console.WriteLine($"\n🔍 Testing: {testCase.Key}");
        Console.WriteLine("-".PadRight(50, '-'));
        
        var testRequest = new { text = testCase.Value };
        
        var testResponse = await httpClient.PostAsJsonAsync("/safety/scan-text", testRequest);
        
        if (testResponse.IsSuccessStatusCode)
        {
            var testResult = await testResponse.Content.ReadFromJsonAsync<JsonElement>();
            
            var isSafe = testResult.GetProperty("isSafe").GetBoolean();
            var severity = testResult.GetProperty("highestSeverity").GetInt32();
            var category = testResult.GetProperty("highestCategory").GetString();
            
            Console.WriteLine($"Is Safe: {(isSafe ? "✅ Yes" : "❌ No")}");
            Console.WriteLine($"Highest Severity: {severity}");
            Console.WriteLine($"Detected Category: {category}");
            
            var flagged = testResult.GetProperty("flaggedCategories");
            if (flagged.GetArrayLength() > 0)
            {
                var flaggedList = string.Join(", ", flagged.EnumerateArray().Select(c => c.GetString()));
                Console.WriteLine($"Flagged Categories: {flaggedList}");
            }
        }
        else
        {
            Console.WriteLine($"⚠️  Request failed: {testResponse.StatusCode}");
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error: {ex.Message}");
    }
}

Console.WriteLine($"\n\n💡 Note: The content safety service helps protect users from harmful content.");

## Step 3: Understanding Severity Levels

Content Safety uses severity levels to categorize content:
- **0**: Safe
- **2**: Low severity
- **4**: Medium severity
- **6**: High severity

In [ ]:
// Helper function to interpret severity
string InterpretSeverity(int severity)
{
    return severity switch
    {
        0 => "✅ Safe",
        2 => "⚠️  Low Risk",
        4 => "🟠 Medium Risk",
        6 => "🔴 High Risk",
        _ => $"❓ Unknown ({severity})"
    };
}

Console.WriteLine("📊 Content Safety Severity Scale:");
Console.WriteLine("=================================");
Console.WriteLine($"0 - {InterpretSeverity(0)}");
Console.WriteLine($"2 - {InterpretSeverity(2)}");
Console.WriteLine($"4 - {InterpretSeverity(4)}");
Console.WriteLine($"6 - {InterpretSeverity(6)}");

// Test a borderline case
Console.WriteLine($"\n\n🧪 Testing Borderline Content:");
var borderlineRequest = new
{
    text = "I'm really angry and frustrated with this situation. I just want to yell at someone!"
};

var borderlineResponse = await httpClient.PostAsJsonAsync("/safety/scan-text", borderlineRequest);
var borderlineResult = await borderlineResponse.Content.ReadFromJsonAsync<JsonElement>();

var borderlineSeverity = borderlineResult.GetProperty("highestSeverity").GetInt32();
Console.WriteLine($"Text: {borderlineRequest.text}");
Console.WriteLine($"Severity: {borderlineSeverity} - {InterpretSeverity(borderlineSeverity)}");
Console.WriteLine($"Is Safe: {borderlineResult.GetProperty("isSafe").GetBoolean()}");

## Step 4: Content Safety in Chat API

The Chat API automatically filters both input and output using content safety.

In [ ]:
Console.WriteLine("🤖 Testing Content Safety in Chat API");
Console.WriteLine("======================================\n");

// Test 1: Safe message (should work)
Console.WriteLine("✅ Test 1: Safe Message");
Console.WriteLine("-".PadRight(50, '-'));

var safeChatRequest = new
{
    message = "What are some healthy breakfast options?",
    agents = new[] { "azure_openai_agent" }
};

try
{
    var safeChatResponse = await httpClient.PostAsJsonAsync("/chat", safeChatRequest);
    
    if (safeChatResponse.IsSuccessStatusCode)
    {
        var safeChatResult = await safeChatResponse.Content.ReadFromJsonAsync<JsonElement>();
        var content = safeChatResult.GetProperty("content").GetString();
        var truncated = content.Length > 150 ? content.Substring(0, 150) + "..." : content;
        
        Console.WriteLine($"✅ Message accepted and processed");
        Console.WriteLine($"Response: {truncated}");
    }
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error: {ex.Message}");
}

// Test 2: Unsafe message (should be blocked)
Console.WriteLine($"\n\n❌ Test 2: Unsafe Message");
Console.WriteLine("-".PadRight(50, '-'));

var unsafeChatRequest = new
{
    message = "Tell me how to hurt someone badly.",
    agents = new[] { "azure_openai_agent" }
};

try
{
    var unsafeChatResponse = await httpClient.PostAsJsonAsync("/chat", unsafeChatRequest);
    
    if (unsafeChatResponse.IsSuccessStatusCode)
    {
        Console.WriteLine($"⚠️  Message was processed (unexpected)");
    }
    else
    {
        var errorContent = await unsafeChatResponse.Content.ReadAsStringAsync();
        Console.WriteLine($"🛡️  Message blocked by content safety");
        Console.WriteLine($"Status: {unsafeChatResponse.StatusCode}");
        
        try
        {
            var errorJson = JsonSerializer.Deserialize<JsonElement>(errorContent);
            if (errorJson.TryGetProperty("detail", out var detail))
            {
                Console.WriteLine($"Message: {detail.GetString()}");
            }
        }
        catch { }
    }
}
catch (Exception ex)
{
    Console.WriteLine($"🛡️  Message blocked: {ex.Message}");
}

Console.WriteLine($"\n💡 The Chat API automatically filters unsafe content in both directions:");
Console.WriteLine($"   - User messages are scanned before processing");
Console.WriteLine($"   - Agent responses are scanned before returning");

## Step 5: Scan Image for Safety (Optional)

If you have an image file, you can test image content safety scanning.

In [ ]:
// Image scanning example (requires an actual image file)
Console.WriteLine("🖼️  Image Content Safety Scanning");
Console.WriteLine("=================================\n");

// Example: You would need to provide an actual image file path
var imagePath = @"C:\path\to\your\image.jpg";  // Update this path

if (File.Exists(imagePath))
{
    try
    {
        // Create multipart form data
        using var form = new MultipartFormDataContent();
        using var fileStream = File.OpenRead(imagePath);
        using var fileContent = new StreamContent(fileStream);
        
        fileContent.Headers.ContentType = new MediaTypeHeaderValue("image/jpeg");
        form.Add(fileContent, "file", Path.GetFileName(imagePath));
        
        Console.WriteLine($"📤 Scanning image: {Path.GetFileName(imagePath)}");
        
        var imageResponse = await httpClient.PostAsync("/safety/scan-image", form);
        imageResponse.EnsureSuccessStatusCode();
        
        var imageResult = await imageResponse.Content.ReadFromJsonAsync<JsonElement>();
        
        Console.WriteLine($"\n🛡️  Image Safety Results:");
        Console.WriteLine($"Is Safe: {imageResult.GetProperty("isSafe").GetBoolean()}");
        Console.WriteLine($"Highest Severity: {imageResult.GetProperty("highestSeverity").GetInt32()}");
        Console.WriteLine($"Highest Category: {imageResult.GetProperty("highestCategory").GetString()}");
        
        var imageCategorySeverities = imageResult.GetProperty("categorySeverities");
        Console.WriteLine($"\n📊 Category Severities:");
        foreach (var prop in imageCategorySeverities.EnumerateObject())
        {
            Console.WriteLine($"   {prop.Name}: {prop.Value.GetInt32()}");
        }
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error scanning image: {ex.Message}");
    }
}
else
{
    Console.WriteLine($"ℹ️  No image file provided or file not found at: {imagePath}");
    Console.WriteLine($"   To test image scanning, update the imagePath variable with a valid image file.");
    Console.WriteLine($"\n   The API endpoint supports:");
    Console.WriteLine($"   - JPEG/JPG images");
    Console.WriteLine($"   - PNG images");
    Console.WriteLine($"   - Maximum file size: 50MB");
}

## Step 6: Batch Testing Multiple Texts

In [ ]:
// Batch test multiple texts
var testTexts = new[]
{
    "I love learning new things about technology!",
    "Can you help me with my homework?",
    "What's the weather like today?",
    "Tell me about the history of computers.",
    "I'm feeling frustrated with this code."
};

Console.WriteLine("🧪 Batch Content Safety Testing");
Console.WriteLine("================================\n");

var safeCount = 0;
var flaggedCount = 0;

foreach (var text in testTexts)
{
    try
    {
        var batchRequest = new { text = text };
        var batchResponse = await httpClient.PostAsJsonAsync("/safety/scan-text", batchRequest);
        var batchResult = await batchResponse.Content.ReadFromJsonAsync<JsonElement>();
        
        var isSafe = batchResult.GetProperty("isSafe").GetBoolean();
        var severity = batchResult.GetProperty("highestSeverity").GetInt32();
        
        var truncatedText = text.Length > 50 ? text.Substring(0, 50) + "..." : text;
        var status = isSafe ? "✅" : "⚠️ ";
        
        Console.WriteLine($"{status} [{severity}] {truncatedText}");
        
        if (isSafe)
            safeCount++;
        else
            flaggedCount++;
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error: {ex.Message}");
    }
}

Console.WriteLine($"\n📊 Batch Test Summary:");
Console.WriteLine($"   Total Tests: {testTexts.Length}");
Console.WriteLine($"   Safe: {safeCount}");
Console.WriteLine($"   Flagged: {flaggedCount}");

## Summary

In this notebook, you learned:
- ✅ How to scan text for unsafe content using the Safety API
- ✅ Understanding severity levels and categories
- ✅ How content safety is automatically applied in the Chat API
- ✅ Testing different types of potentially unsafe content
- ✅ How to scan images for inappropriate content
- ✅ Batch testing multiple texts

## Key Takeaways
- Content safety is a critical feature for production AI applications
- The framework automatically filters both user input and agent output
- Multiple severity levels allow for nuanced content filtering
- Both text and images can be analyzed for safety

## Safety Categories
The Azure Content Safety service checks for:
- **Hate**: Hateful or discriminatory content
- **Violence**: Violent or graphic content
- **Self-Harm**: Content related to self-injury
- **Sexual**: Sexually explicit content

## Next Steps
- Try the **Memory Demo** to learn about session management and conversation history